### Programa para calcular $t_{\mathrm{obs}}$ y $S/N$ de las observaciones

Importamos los paquetes que vamos a usar

In [ ]:
import pypulse as pulsar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

Definimos para qué antena vamos a tomar las observaciones.

In [9]:
ant = "A2"

Analizamos las observaciones usando PyPulse

In [ ]:
# Creamos una lista con todos los archivos .pfd de observaciones

os.chdir('./pfds/')       # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*' + ant + '*pfd')                           # lista de archivos .pfd
print("Numero de observaciones = " + str(len(pfd_files)))

# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])
    
# Save all PSRFITS files
psrfits_files = glob.glob('*' + ant + '*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))

# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    # First crunch in time and frequency
    observation.fscrunch()
    observation.tscrunch()
    
    # Now get array of the best profile
    tmp_singlepulse = observation.getSinglePulses(windowsize=256)
    
    # Align and normalize
    tmp_singlepulse.center_align()
    tmp_singlepulse.normalize()
    
    # Save aligned and normalize in each observation
    single_pulses.append(tmp_singlepulse)
    
# Save single pulse object
sp_observations = dict(zip(psrfits_files, single_pulses))

os.chdir('..')                                 # salimos del directorio que contiene los .pfds

Calculamos y guardamos los valores de $t_{\mathrm{obs}}$ y $S/N$

In [ ]:
files = []                                            # listado de observaciones
list = open("./results_" + ant + ".txt", "w+")        # archivo que contendrá S/N y t_obs
    
for key in observations:
        
    time = (observations[key].getDuration())/60                        # calculamos el tiempo de integración en minutos
    sn_obs = sp_observations[key].getSN()                              # calculamos el S/N de la observación
    file = key.replace(".fits", ".pfd")
    list.write(file + "   " + str(sn_obs) + "   " + str(time) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
        
list.close()                                                       # cerramos el archivo de salida

Una vez que tenemos los datos para las observaciones de ambas antenas, podemos juntarlos en un único archivo

In [ ]:
filenames = ['results_A1.txt', 'results_A2.txt', ...]
with open("./results_total.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
